# Interval-Based Time Series Classification

Interval-based methods build ensembles of features extracted from random sub-intervals of the
series. Each tree sees statistics like mean, variance, or slope over randomly sampled windows.

## Core idea
Given a series $x_{1:T}$, sample intervals $[s_k, e_k]$ and compute feature vectors
$\phi_k(x) = [\mu_k, \sigma_k, 	ext{slope}_k, \dots]$. An ensemble learns a classifier from these
interval features. This gives robustness to local patterns without full alignment.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

np.random.seed(11)
t = np.linspace(0, 4 * np.pi, 120)
x = np.sin(t) + 0.3 * np.sin(3 * t) + 0.2 * np.random.randn(len(t))

# Sample random intervals
n_intervals = 6
intervals = []
for _ in range(n_intervals):
    start = np.random.randint(0, len(x) - 10)
    end = np.random.randint(start + 5, min(start + 30, len(x)))
    intervals.append((start, end))

fig = go.Figure()
fig.add_trace(go.Scatter(y=x, mode='lines', name='series'))
for start, end in intervals:
    fig.add_vrect(x0=start, x1=end, fillcolor='orange', opacity=0.2, line_width=0)
fig.update_layout(title='Random intervals used for feature extraction', xaxis_title='time index')
fig.show()

rows = []
for idx, (start, end) in enumerate(intervals, start=1):
    segment = x[start:end]
    slope = np.polyfit(np.arange(len(segment)), segment, 1)[0]
    rows.append({
        'interval': f'I{idx} [{start},{end})',
        'mean': segment.mean(),
        'std': segment.std(ddof=0),
        'slope': slope,
        'min': segment.min(),
        'max': segment.max(),
    })

features_df = pd.DataFrame(rows)
features_df


## sktime inventory for interval-based classifiers
Interval-based algorithms often live in modules with `interval_based` in the path.

In [ ]:
try:
    import pandas as pd
    from sktime.registry import all_estimators

    def _estimators_df():
        try:
            return all_estimators(estimator_types='classifier', as_dataframe=True)
        except TypeError:
            ests = all_estimators(estimator_types='classifier')
            return pd.DataFrame(
                [{'name': n, 'class': c.__name__, 'module': c.__module__} for n, c in ests]
            )

    df = _estimators_df()
    mask = df['module'].str.contains('interval', case=False, na=False)
    df_interval = df[mask].sort_values('name')
    df_interval
except Exception as exc:
    print('sktime not installed or registry unavailable:', exc)


## Minimal sktime example (optional)
TimeSeriesForestClassifier is a classic interval-based ensemble.

In [ ]:
try:
    from sktime.datasets import load_basic_motions
    from sktime.classification.interval_based import TimeSeriesForestClassifier
    from sklearn.metrics import classification_report

    X_train, y_train = load_basic_motions(split='train', return_X_y=True)
    X_test, y_test = load_basic_motions(split='test', return_X_y=True)

    clf = TimeSeriesForestClassifier(n_estimators=200, random_state=42)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print(classification_report(y_test, pred))
except Exception as exc:
    print('Install sktime to run the interval-based demo:', exc)


## When to use
- Strong for capturing local discriminative patterns without alignment.
- Ensemble size matters; more intervals increase accuracy but also cost.
- Works well when class differences appear in short segments.